На входе есть множество характеристик, расчитанных по изображениям специальной камеры, снимающей небосвод. Наша задача: предсказать значение приходящей коротковолновой радиации по имеющемуся датасету с характеристиками этих изображений. Подробнее: https://github.com/MKrinitskiy/Sirius-AI4Climate-2024/blob/main/DASIO-dataset-description.md

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv("/content/drive/MyDrive/dataset_25perc.csv", sep=',')
df

,photo_name,photo_datetime,CM3up[W/m2],CG3up[W/m2],CM3down[W/m2],CG3down[W/m2],radiation_datetime,feature0,feature1,feature2,...,feature157,feature158,feature159,feature160,feature161,date-hour,datetime_UTC,lat,lon,sun_altitude
0,img-2021-08-30T15-34-05devID1.jpg,2021-08-30 15:34:05,50.891321,-50.038394,27.577978,-4.928024,2021-08-30 15:34:09,91.442066,624.291578,255.0,...,0.431572,0.456165,0.480924,0.527055,0.668962,2021-08-30-15,2021-08-30 15:34:04.576216,70.211377,36.001765,8.445168
1,img-2017-03-13T08-26-03devID2.jpg,2017-03-13 08:26:03,523.223433,-101.593103,177.029772,16.016077,2017-03-13 08:26:10,88.081871,1268.517161,255.0,...,0.398634,0.445029,0.517948,0.608639,0.875906,2017-03-13-08,2017-03-13 08:26:02.612637,35.777146,14.896456,35.690805
2,img-2021-09-01T16-57-43devID2.jpg,2021-09-01 16:57:43,13.078217,-32.885081,-0.284309,-1.800624,2021-09-01 16:57:49,82.160335,321.809289,118.0,...,0.419772,0.432900,0.445461,0.458523,0.472848,2021-09-01-16,2021-09-01 16:57:43.252118,71.272493,20.304985,6.063512
3,img-2017-03-03T11-53-01devID1.jpg,2017-03-03 11:53:01,698.736891,-69.087101,217.970276,33.453699,2017-03-03 11:53:04,111.489619,1925.598648,255.0,...,0.553571,0.612063,0.682985,0.798806,0.984047,2017-03-03-11,2017-03-03 11:53:00.718105,18.085080,40.184646,47.145397
4,img-2021-08-09T01-42-51devID2.jpg,2021-08-09 01:42:51,7.202496,-52.123327,7.107726,-0.852927,2021-08-09 01:42:59,83.349643,679.059381,255.0,...,0.436287,0.465932,0.501567,0.570519,0.777904,2021-08-09-01,2021-08-09 01:42:51.232329,64.703417,40.527603,4.120047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260428,img-2018-06-27T17-30-43devID2.jpg,2018-06-27 17:30:43,322.311700,-90.125971,27.577978,-12.130520,2018-06-27 17:30:49,89.302117,1348.725930,255.0,...,0.441107,0.469089,0.533467,0.654001,0.980260,2018-06-27-17,2018-06-27 17:30:42.751351,59.097073,1.089170,23.386556
260429,img-2019-12-18T18-04-24devID1.jpg,2019-12-18 18:04:24,81.501929,-89.841662,28.715215,-10.329896,2019-12-18 18:04:24,91.912472,887.573445,255.0,...,0.419275,0.443204,0.485278,0.572748,0.795019,2019-12-18-18,2019-12-18 18:04:24.457886,15.314597,-22.494321,11.863466
260430,img-2017-03-07T14-35-42devID2.jpg,2017-03-07 14:35:42,202.238509,-112.775926,132.677560,0.758157,2017-03-07 14:35:45,89.728605,1632.637611,255.0,...,0.433466,0.486971,0.561415,0.682751,0.895019,2017-03-07-14,2017-03-07 14:35:42.473677,29.925779,32.540550,15.615127
260431,img-2017-01-23T04-19-14devID2.jpg,2017-01-23 04:19:14,30.515839,-4.833254,3.885557,0.947697,2017-01-23 04:19:14,82.498614,459.167360,171.0,...,0.402658,0.426453,0.455370,0.500498,0.570521,2017-01-23-04,2017-01-23 04:19:13.461347,6.298152,95.153380,55.749331


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260433 entries, 0 to 260432
Columns: 172 entries, CM3up[W/m2] to sin
dtypes: float64(171), int32(1)
memory usage: 340.8 MB


In [46]:
df.describe()

,CM3up[W/m2],CG3up[W/m2],CM3down[W/m2],CG3down[W/m2],feature0,feature1,feature2,feature3,feature4,feature5,...,feature158,feature159,feature160,feature161,lat,lon,sun_altitude,days,cos,sin
count,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,...,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000,260433.000000
mean,267.162010,-47.479956,65.454870,2.622060,94.951865,953.031869,205.861630,19.476226,0.095298,0.785111,...,0.482219,0.522926,0.591220,0.718453,44.573726,15.619780,27.891180,15.819689,-0.280462,-0.339077
std,268.825122,37.967731,72.174757,15.449046,14.169739,713.662483,61.633465,11.989895,0.925641,1.559171,...,0.073023,0.092471,0.135947,0.214833,35.062012,43.859881,17.799010,9.010917,0.672813,0.594721
min,5.022793,-210.293932,-106.710666,-172.101749,13.957575,69.770570,85.000000,0.000000,-2.751973,-1.793168,...,0.262700,0.262700,0.288733,0.322899,-49.639077,-63.469069,-0.605456,1.000000,-0.999963,-0.999991
25%,58.757205,-78.469300,16.300386,-4.264636,85.719352,287.648025,131.000000,11.000000,-0.833563,-0.464717,...,0.428845,0.444266,0.462479,0.486141,19.756974,-25.966896,13.507289,8.000000,-0.865307,-0.826354
50%,161.298005,-37.339256,41.035274,-0.758157,90.536358,786.758960,255.000000,19.000000,-0.051501,-0.004458,...,0.462577,0.507011,0.571519,0.728450,59.499908,10.493023,25.086516,16.000000,-0.618671,-0.559589
75%,404.382248,-13.078217,87.946268,3.222169,99.438184,1495.766397,255.000000,28.000000,1.028095,1.949433,...,0.517514,0.581622,0.698805,0.963546,69.496262,57.963884,39.702290,24.000000,0.373720,0.008607
max,1425.904687,21.228410,624.153148,111.543920,207.011743,10271.560471,255.000000,94.000000,2.907309,10.901657,...,0.984047,0.984047,0.984047,0.986706,83.001788,102.269799,87.713263,31.000000,0.994671,0.989932


In [5]:
df = df.drop('photo_name', axis = 1)

In [6]:
df['photo_datetime'] = pd.to_datetime(df['photo_datetime'])

In [7]:
df['days'] = df['photo_datetime'].dt.day
df['year_date'] = df['photo_datetime'].dt.year

In [8]:
# Вычисляем разницу в секундах между значениями в столбце 'photo_datetime' и начальной датой
df['date_seconds'] = (df['photo_datetime'] - df['photo_datetime'].min()).dt.total_seconds()

In [9]:
df = df.sort_values("date_seconds", axis=0)

In [10]:
df.index=range(0,len(df))

In [11]:
df['dateyear'] = (df['photo_datetime'].dt.dayofyear - 1) / 365
df['cos'] = np.cos(df['dateyear'] * 2 * np.pi)
df['sin'] = np.sin(df['dateyear'] * 2 * np.pi)

In [59]:
# Вывод важных полей
df[['CM3up[W/m2]', 'CG3up[W/m2]', 'sin', 'cos', 'lat', 'lon', 'sun_altitude','feature0','feature1','feature2', 'days']]


,CM3up[W/m2],CG3up[W/m2],sin,cos,lat,lon,sun_altitude,feature0,feature1,feature2,days
0,143.102225,-88.135807,-0.967938,-0.251190,66.584686,-29.280270,14.488320,96.890459,1671.656306,255.0,17
1,143.102225,-88.135807,-0.967938,-0.251190,66.584686,-29.280270,14.488320,108.081154,1784.724755,255.0,17
2,159.023532,-87.377650,-0.967938,-0.251190,66.586937,-29.289083,14.655731,98.760528,1723.527972,255.0,17
3,159.023532,-87.377650,-0.967938,-0.251190,66.586937,-29.289083,14.655731,109.894952,1814.364325,255.0,17
4,160.824156,-86.903802,-0.967938,-0.251190,66.588477,-29.294644,14.739936,111.069528,1877.696783,255.0,17
...,...,...,...,...,...,...,...,...,...,...,...
260428,182.242105,-52.312866,-0.903356,-0.428892,55.545024,12.705800,30.696305,93.887588,205.434115,135.0,6
260429,181.483948,-52.123327,-0.903356,-0.428892,55.544092,12.705888,30.642495,93.766927,205.713884,131.0,6
260430,180.915329,-52.028557,-0.903356,-0.428892,55.542581,12.706041,30.552547,93.492497,209.849320,133.0,6
260431,180.725790,-52.028557,-0.903356,-0.428892,55.542266,12.706072,30.534529,94.056191,205.642272,144.0,6


## Задача регрессии

In [13]:
df = df.loc[:, ~df.columns.str.contains('date')]

In [14]:
X = df.drop(['CM3up[W/m2]','CG3up[W/m2]','CM3down[W/m2]','CG3down[W/m2]'], axis = 1)
y = df['CM3up[W/m2]']

In [15]:
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=2, test_size=0.2, random_state=42)

for train_index, test_index in gss.split(X, y, groups=df['days']):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [16]:
# Проверка пересекающихся данных в столбцах 'date_days' и 'year_date'
print("Пересечение данных в X_train и X_test:")
X_test["days"].unique()

Пересечение данных в X_train и X_test:


array([21, 22, 14, 29,  1,  6,  8], dtype=int32)

In [17]:
X_train = X_train.loc[:, ~X_train.columns.str.contains('date')]
X_test = X_test.loc[:, ~X_test.columns.str.contains('date')]
y_train = y_train
y_test = y_test

In [18]:
!pip install -r '/content/drive/MyDrive/Colab Notebooks/Sirius project/requirements.txt'

In [19]:
!pip install catboost

In [20]:
!pip install numpy==1.24.0

In [21]:
#from sklearn.linear_model import CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV

In [22]:
# Задаем наборы параметров для подбора лучших
random_params = {'iterations': [100, 500, 1000],  # Количество итераций
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Скорость обучения
    'l2_leaf_reg': [1, 3, 5, 10],  # Регуляризация L2
    'bagging_temperature': [0.0, 0.2, 0.5, 1.0],  # Температура для бэггинга
    'random_strength': [0.0, 0.1, 0.5],  # Сила случайности
    'max_depth': [6, 8, 10],  # Максимальная глубина деревьев
    'subsample': [0.7, 0.8, 0.9, 1.0],  # Размер подвыборки
    'colsample_bylevel': [0.5, 0.7, 1.0]  # Размер подвыборки по уровням
                 }

In [23]:
# Устраняем проблемы с форматом имен признаков для обучения
X_train.columns = X_train.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_').str.replace('>', '_')
X_test.columns = X_test.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_').str.replace('>', '_')

In [24]:
from catboost import CatBoostRegressor

# Создаем модель
catboost = CatBoostRegressor(task_type='CPU', verbose=0)

model = RandomizedSearchCV(catboost, param_distributions=random_params, n_iter=10,
                                   cv=3,
                                   verbose=1,
                                   n_jobs=-1,
                                   random_state=42)

# Обучаем
model.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7be9cce86990>,
                   n_jobs=-1,
                   param_distributions={'bagging_temperature': [0.0, 0.2, 0.5,
                                                                1.0],
                                        'border_count': [32, 64, 128],
                                        'colsample_bylevel': [0.5, 0.7, 1.0],
                                        'iterations': [100, 500, 1000],
                                        'l2_leaf_reg': [1, 3, 5, 10],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [6, 8, 10],
                                        'random_strength': [0.0, 0.1, 0.5],
                                        'subsample': [0.7, 0.8, 0.9, 1.0]},
                   random_state=42, verbose=1)

In [35]:
# Лучшие параметры
model.best_params_

{'subsample': 0.8,
 'random_strength': 0.0,
 'max_depth': 6,
 'learning_rate': 0.1,
 'l2_leaf_reg': 10,
 'iterations': 500,
 'colsample_bylevel': 0.5,
 'border_count': 64,
 'bagging_temperature': 0.0}

In [25]:
preds = model.predict(X_test)

In [26]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error

In [27]:
mean_absolute_percentage_error(preds,y_test)

3.586126468324626

In [38]:
r2_score(preds,y_test)

0.9452663284535033

In [42]:
mean_absolute_error(preds,y_test)

31.891468327139986

In [37]:
root_mean_squared_error(preds,y_test)

63.48254929657965

In [39]:
preds_series = pd.Series(preds)
preds_series.index = y_test.index
preds_series.name = 'CG3up_predicted'
preds_series

,CG3up_predicted
664,78.411943
665,62.613659
666,61.563994
667,49.009527
668,17.348684
...,...
260428,157.324602
260429,154.108743
260430,154.217545
260431,157.175387


In [40]:
results = pd.merge(y_test, preds_series, left_index=True, right_index=True, how='outer')

In [43]:
results.sample(5)

,CM3up[W/m2],CG3up_predicted
226414,29.283833,30.396767
250472,86.903802,108.070768
165152,273.031464,290.166764
223934,61.505526,70.151759
75817,843.260661,826.241192


# Сохранение/использование модели

In [53]:
from joblib import dump, load
dump(model, 'shortRmodel.joblib')

['shortRmodel.joblib']

In [54]:
CM3up = load('shortRmodel.joblib')

In [55]:
import time

start_time = time.time()
num_samples = len(X_test)
preds = CM3up.predict(X_test)

total_time = time.time() - start_time
average_time = total_time / num_samples

print(f"Общее время выполнения: {total_time:.6f} секунд")
print(f"Среднее время выполнения: {average_time:.6f} секунд")


Общее время выполнения: 0.221330 секунд
Среднее время выполнения: 0.000004 секунд


In [56]:
mae = mean_absolute_error(preds,y_test)
rmse = root_mean_squared_error(preds, y_test)
print('mae:', mae, 'rmse:', rmse)

mae: 31.891468327139986 rmse: 63.48254929657965


In [57]:
std = df.describe().loc['std', 'CM3up[W/m2]']
print('std:', std)

std: 268.8251224422064


**Ошибка в целом адекватна относительно стандартного отклонения во входных данных**